<a href="https://colab.research.google.com/github/nobuhikosekiya/qiitablog_appsearch_sample/blob/main/qiitablog_appsearch_index_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.初期設定

## ライブラリの有効化

In [3]:
%pip install -q elasticsearch requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.9 MB/s eta 0:00:00


In [4]:
from pprint import pprint
import os, json
import requests
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk

## Elasticsearchの設定

In [6]:
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_API_KEY = getpass("Elastic deployment API Key")
if ELASTIC_CLOUD_ID == '':
  ELASTIC_URL = getpass("Elastic deployment URL. No need if Cloud ID is provided.")
if ELASTIC_API_KEY == '':
  ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
  ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")

if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    # request_timeout=300,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

Elastic deployment Cloud ID··········
Elastic deployment API Key··········
ObjectApiResponse({'name': 'instance-0000000054', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


## Elasticsearchで使うインデックス名の設定

In [7]:
INDEX_NAME=input("Elasticsearchのインデックスの名前 (空入力はqiita-blog-appsearchになります):")
if INDEX_NAME == '':
    INDEX_NAME='qiita-blog-appsearch'
INDEX_NAME

Elasticsearchのインデックスの名前 (空入力はqiita-blog-appsearchになります):test-appsearch-01


'test-appsearch-01'

# 2.検索ドキュメントの準備
この例では、QiitaのElasticsearch Japan所属の記事を検索対象としますので、その記事のダウンロードをQiita APIを使って行います。

## Qiita記事のダウンロード

In [14]:
# h = {'Authorization': 'Bearer xxxx'} # ユーザ認証する場合
h = {}
url = "https://qiita.com/api/v2/items?"

# tag別に記事をPAGEだけ繰り返し取得
query = "&query=org%3Aelasticsearch_japan"
# 検索で指定した期間内に作成された記事数を取得
res = requests.get(url=url + query, headers=h)
# サーバーからの応答
print(res.status_code, res.reason)
# print("指定しているタグ: " + tag_name)
total_count = int(res.headers['Total-Count'])
print("total_count: " + str(total_count))

page = f"page=1&per_page={total_count}"
os.makedirs("qiita-downloads", exist_ok=True)
res = requests.get(url=url + page + query, headers=h)
documents = json.loads(res.text)
for doc in documents:
    search_doc = {
        "title": doc["title"],
        "url": doc["url"],
        "body": doc["body"],
        "tags": doc["tags"],
        "created_at": doc["created_at"],
        "updated_at": doc["updated_at"],
        "id": doc["id"],
        "likes_count": doc["likes_count"],
        "reactions_count": doc["reactions_count"],
        "stocks_count": doc["stocks_count"],
        "page_views_count": doc["page_views_count"],
        "organization_url_name": doc["organization_url_name"],
        "comments_count": doc["comments_count"]
    }
    title = doc["title"].replace('/', '_')
    filename = "./qiita-downloads" + "/" + title + ".json"
    with open(filename, 'w') as f:
        json.dump(search_doc, f, indent=2, ensure_ascii=False)

200 OK
total_count: 74


In [15]:
DRIVE_FOLDER = "./qiita-downloads/"
json_docs = []

# 指定されたフォルダ内のファイルを走査
for filename in os.listdir(DRIVE_FOLDER):
    # ファイルがJSONファイルであることを確認
    if filename.endswith(".json"):
        file_path = os.path.join(DRIVE_FOLDER, filename)
        # JSONファイルを開いてデータを読み取り、dictオブジェクトとして追加
        with open(file_path, "r", encoding="utf-8") as json_file:
            json_data = json.load(json_file)
            tags_names = []
            for tag in json_data['tags']:
                tags_names.append(tag['name'])
            json_data['tags'] = tags_names
            json_docs.append(json_data)

# 全てのJSONドキュメントがjson_docsリストに格納されました
print(f'document count: {len(json_docs)}')
print(json_docs[0] if len(json_docs) > 0 else None)

document count: 74
{'title': 'Elasticsearchのマシン・ラーニング異常検知の動きを理解する(3) [変更設定編]', 'url': 'https://qiita.com/nobuhikosekiya/items/ee8be363fe6e8faa75f1', 'body': 'シリーズ記事\n[Elasticsearchのマシン・ラーニング異常検知の動きを理解しよう(1)](https://qiita.com/nobuhikosekiya/items/e0fbb7792a9b70284c57)\n[Elasticsearchのマシン・ラーニング異常検知の動きを理解する(2) [アラート編]](https://qiita.com/nobuhikosekiya/items/59cb1705dac98860c5dc)\n[Elasticsearchのマシン・ラーニング異常検知の動きを理解する(3) [変更設定編]](https://qiita.com/nobuhikosekiya/items/ee8be363fe6e8faa75f1)\n\n# はじめに\n[前回](https://qiita.com/nobuhikosekiya/items/59cb1705dac98860c5dc)の記事では、MLの異常検知でアラートを出すための手順について書きました。\n異様検知を運用で真剣に使い始めると、色々な変更作業が必要になってきます。\n\n# どんな変更が想定されるか？\n全てではないかもしれませんが、運用中の主な変更作業についてまとめます。\n\n### 異常検知ジョブの分析対象の指標やBuket spanの変更(Detector設定)\nSingle Metricタイプのジョブは検知対象の指標が一つなので、変更したいとしたら指標の集約期間であるBucket spanくらいかと思います。\nしかし、Multi Metricタイプのジョブは検知対象の指標及びInfluencer、Split Fieldも複数指定したりと、途中で変更したくなる可能性があります。\nこれらは学習モデルに影響のあるパラメータなので、基本的にはジョブ作成時にしか設定できません。変更したい場合はジョブをクローンして、その際に設定を変更する方法があります。\n詳しい

以下のように、ダウンロードしたJSONには以下のキーが含まれています。このキー名を元に後続ステップでElasticsearchのインデックススキーマを定義していきます。

body, comments_count, created_at, id,likes_count, organization_url_name, page_views_count, reactions_count', stocks_count, tags, title, updated_at, url

In [17]:
pprint(json_docs[0])

{'body': 'シリーズ記事\n'
         '[Elasticsearchのマシン・ラーニング異常検知の動きを理解しよう(1)](https://qiita.com/nobuhikosekiya/items/e0fbb7792a9b70284c57)\n'
         '[Elasticsearchのマシン・ラーニング異常検知の動きを理解する(2) '
         '[アラート編]](https://qiita.com/nobuhikosekiya/items/59cb1705dac98860c5dc)\n'
         '[Elasticsearchのマシン・ラーニング異常検知の動きを理解する(3) '
         '[変更設定編]](https://qiita.com/nobuhikosekiya/items/ee8be363fe6e8faa75f1)\n'
         '\n'
         '# はじめに\n'
         '[前回](https://qiita.com/nobuhikosekiya/items/59cb1705dac98860c5dc)の記事では、MLの異常検知でアラートを出すための手順について書きました。\n'
         '異様検知を運用で真剣に使い始めると、色々な変更作業が必要になってきます。\n'
         '\n'
         '# どんな変更が想定されるか？\n'
         '全てではないかもしれませんが、運用中の主な変更作業についてまとめます。\n'
         '\n'
         '### 異常検知ジョブの分析対象の指標やBuket spanの変更(Detector設定)\n'
         'Single Metricタイプのジョブは検知対象の指標が一つなので、変更したいとしたら指標の集約期間であるBucket '
         'spanくらいかと思います。\n'
         'しかし、Multi Metricタイプのジョブは検知対象の指標及びInfluencer、Split '
         'Fieldも複数指定したりと、途中で変更したくなる可能性があります。\n'
         'これらは学習モデル

# 3.Elasticsearchインデックスの作成


まずElasticsearchのインデックスのベースを作成します。

In [8]:
if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 1
      }
  }
)


Index 'test-appsearch-01' does not exist.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-appsearch-01'})

Kuromojiアナライザをインデックスのsettingsに追加します。

In [9]:
es.indices.close(index=INDEX_NAME)

add_settings = {
  "index": {
    "analysis": {
      "char_filter": {
        "normalize": {
          "mode": "compose",
          "name": "nfkc",
          "type": "icu_normalizer"
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "tokenizer": {
        "ja_kuromoji_tokenizer": {
          "mode": "search",
          "discard_compound_token": "true",
          "type": "kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

# Define the new settings you want to apply
add_settings = {
  "index": {
    "analysis": {
      "analyzer": {
        "ja_kuromoji_index_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        },
        "ja_kuromoji_search_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

es.indices.open(index=INDEX_NAME, request_timeout=60)


<ipython-input-9-d31898a008a3>:76: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.open(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

App Searchが使うアナライザの設定を追加します。

In [10]:
es.indices.close(index=INDEX_NAME)
add_settings = {
  "index": {
    "analysis": {
        "filter": {
          "front_ngram": {
            "type": "edge_ngram",
            "min_gram": "1",
            "max_gram": "12"
          },
          "bigram_joiner": {
            "max_shingle_size": "2",
            "token_separator": "",
            "output_unigrams": "false",
            "type": "shingle"
          },
          "bigram_max_size": {
            "type": "length",
            "max": "16",
            "min": "0"
          },
          "bigram_joiner_unigrams": {
            "max_shingle_size": "2",
            "token_separator": "",
            "output_unigrams": "true",
            "type": "shingle"
          },
          "delimiter": {
            "split_on_numerics": "true",
            "generate_word_parts": "true",
            "preserve_original": "false",
            "catenate_words": "true",
            "generate_number_parts": "true",
            "catenate_all": "true",
            "split_on_case_change": "true",
            "type": "word_delimiter_graph",
            "catenate_numbers": "true",
            "stem_english_possessive": "true"
          },
          "ja-stop-words-filter": {
            "type": "stop",
            "stopwords": "_japanese_"
          },
          "ja-stem-filter": {
            "type": "kuromoji_stemmer",
            "minimum_length": 4
          }
        },
        "analyzer": {
          "i_prefix": {
            "filter": [
              "front_ngram"
            ],
            "tokenizer": "kuromoji_tokenizer"
          },
          "iq_text_delimiter": {
            "filter": [
              "delimiter",
              "kuromoji_baseform",
              "kuromoji_part_of_speech",
              "cjk_width",
              "ja_stop",
              "kuromoji_stemmer",
              "lowercase"
            ],
            "char_filter": [
              "normalize"
            ],
            "tokenizer": "whitespace"
          },
          "q_prefix": {
            "filter": [
              "front_ngram"
            ],
            "tokenizer": "kuromoji_tokenizer"
          },
          "iq_text_base": {
            "filter": [
              "kuromoji_baseform",
              "kuromoji_part_of_speech",
              "cjk_width",
              "ja_stop",
              "kuromoji_stemmer",
              "lowercase"
            ],
            "char_filter": [
              "normalize"
            ],
            "tokenizer": "kuromoji_tokenizer"
          },
          "iq_text_stem": {
            "filter": [
              "kuromoji_baseform",
              "kuromoji_part_of_speech",
              "cjk_width",
              "ja_stop",
              "kuromoji_stemmer",
              "lowercase"
            ],
            "char_filter": [
              "normalize"
            ],
            "tokenizer": "kuromoji_tokenizer"
          },
          "i_text_bigram": {
            "filter": [
              "delimiter",
              "kuromoji_baseform",
              "kuromoji_part_of_speech",
              "cjk_width",
              "ja_stop",
              "kuromoji_stemmer",
              "lowercase",
              "bigram_joiner",
              "bigram_max_size"
            ],
            "char_filter": [
              "normalize"
            ],
            "tokenizer": "kuromoji_tokenizer"
          },
          "q_text_bigram": {
            "filter": [
              "delimiter",
              "kuromoji_baseform",
              "kuromoji_part_of_speech",
              "cjk_width",
              "ja_stop",
              "kuromoji_stemmer",
              "lowercase",
              "bigram_joiner_unigrams",
              "bigram_max_size"
            ],
            "char_filter": [
              "normalize"
            ],
            "tokenizer": "kuromoji_tokenizer"
          }
        }
      }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)
es.indices.open(index=INDEX_NAME, request_timeout=60)

<ipython-input-10-bfe7f6a83c6e>:145: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.open(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

インデックスのマッピングを定義します。JSONドキュメントと照らし合わせながらフィールドを定義していきます。

In [11]:
es.indices.close(index=INDEX_NAME, request_timeout=60)

add_mapping = {
  "properties": {
    "title": {
      "type": "text",
      "search_analyzer": "ja_kuromoji_search_analyzer",
      "analyzer": "ja_kuromoji_index_analyzer"
    },
    "body": {
      "type": "text",
      "search_analyzer": "ja_kuromoji_search_analyzer",
      "analyzer": "ja_kuromoji_index_analyzer"
    },
    "seq_num": {
      "type": "long"
    },
    "source": {
      "type": "keyword"
    },
    "tags": {
      "type": "keyword"
    },
    "url": {
      "type": "keyword"
    },
    "id": {
      "type": "keyword"
    },
    "created_at": {
      "type": "date"
    },
    "updated_at": {
      "type": "date"
    },
    "likes_count": {
      "type": "long"
    },
    "reactions_count": {
      "type": "date"
    },
    "stocks_count": {
      "type": "long"
    },
    "page_views_count": {
      "type": "long"
    },
    "comments_count": {
      "type": "long"
    },
    "organization_url_name": {
      "type": "keyword"
    }
  }
}

es.indices.put_mapping(index=INDEX_NAME, body=add_mapping)

es.indices.open(index=INDEX_NAME)

<ipython-input-11-2341abc7c3ee>:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.close(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

App SearchのPrecision Tuningを使うためのサブフィールドprefix, delimiter, joined, stemの4つをテキスト検索させたいフィールド（titleとbody)に対して追加します。詳しくは以下のドキュメントにあります。

https://www.elastic.co/guide/en/app-search/current/elasticsearch-engines-text-subfields-support-conventions.html


In [13]:
es.indices.close(index=INDEX_NAME, request_timeout=60)

add_mapping = {
  "properties": {
    "title": {
      "type": "text",
      "search_analyzer": "ja_kuromoji_search_analyzer",
      "analyzer": "ja_kuromoji_index_analyzer",
      #---以降追加部分---
      "fields": {
        "prefix": {
          "search_analyzer": "ja_kuromoji_search_analyzer",
          "analyzer": "ja_kuromoji_index_analyzer",
          "type": "text",
          "index_options": "docs"
        },
        "delimiter": {
          "analyzer": "iq_text_delimiter",
          "type": "text",
          "index_options": "freqs"
        },
        "joined": {
          "search_analyzer": "q_text_bigram",
          "analyzer": "i_text_bigram",
          "type": "text",
          "index_options": "freqs"
        },
        "stem": {
          "analyzer": "iq_text_stem",
          "type": "text"
        }
      }
    },
    "body": {
      "type": "text",
      "search_analyzer": "ja_kuromoji_search_analyzer",
      "analyzer": "ja_kuromoji_index_analyzer",
      #---以降追加部分---
      "fields": {
        "prefix": {
          "search_analyzer": "ja_kuromoji_search_analyzer",
          "analyzer": "ja_kuromoji_index_analyzer",
          "type": "text",
          "index_options": "docs"
        },
        "delimiter": {
          "analyzer": "iq_text_delimiter",
          "type": "text",
          "index_options": "freqs"
        },
        "joined": {
          "search_analyzer": "q_text_bigram",
          "analyzer": "i_text_bigram",
          "type": "text",
          "index_options": "freqs"
        },
        "stem": {
          "analyzer": "iq_text_stem",
          "type": "text"
        }
      }
    }
  }
}

es.indices.put_mapping(index=INDEX_NAME, body=add_mapping)

es.indices.open(index=INDEX_NAME)

<ipython-input-13-0295fcb509ac>:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.close(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

# 4. ドキュメントのインジェスト
前のステップでダウンロードしたQiita記事をElasticsearchのインデックスにアップします。

In [16]:
from elasticsearch import Elasticsearch, helpers

# 前の実行で残っているDocumentはクリアしてからインジェストします
if es.indices.exists(index=INDEX_NAME):
    es.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

index_docs = []
for doc in json_docs:
    # doc_json['_run_ml_inference'] = True
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc,
    })

try:
    r = helpers.bulk(es, index_docs)
except Exception as e:
    print(e)
    raise e

response = es.search(index=INDEX_NAME, query={"match_all": {}}, source=["title"])
for hit in response['hits']['hits']:
    print(hit['_source'])

{'title': 'Elasticsearchのマシン・ラーニング異常検知の動きを理解する(3) [変更設定編]'}
{'title': 'ElasticsearchでRAG (Retrieval Augmented Generation) を試す'}
{'title': 'Elasticsearchに日本語のNLPモデルをアップロードする'}
{'title': 'OpenLDAPのログをElastic AgentのCustom Logsで取り込む'}
{'title': 'ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)'}
{'title': 'Elastic CloudでHotに溜まっているデータを別のティアに移動するやり方'}
{'title': 'Elasticsearchにカスタム時系列データを取り込む (Elastic Agent編)'}
{'title': 'Elastic CloudとLangChainでQiitaの記事をベクトル/セマンティック検索する方法の紹介'}
{'title': 'Elastic Stack一式をelastic-packageコマンドを使ってローカルに一発で立ち上げる'}
{'title': 'Upgrade Assistantを使ってElastic Stackをバージョンアップ'}


# 5. App Seachエンジン用のALIASの設定
App Searchからはエイリアス経由でインデックスを使うため、エイリアス名を指定します。ここでは、元のインデックス名に対してプレフィックス"search-"を追加したものをエイリアス名としています。

In [18]:
ALIAS=f"search-{INDEX_NAME}"
es.indices.put_alias(index=INDEX_NAME, name=ALIAS)
ALIAS

'search-test-appsearch-01'